In [4]:
import os, re, csv
import pandas as pd
from kiwipiepy import Kiwi
## pip install kiwipiepy

In [5]:
def classification(df, class_lst=['이중전공', '부전공', '연계전공', '전공심화', '융합전공']):

    """
    데이터 프레임의 형태는 첫 column이 id(인덱스 값), 그리고 세 번째 column(인덱스 번호 2)이 분류가 들어갈 곳이므로 임의의 행이 필요하다.
    또한 '내용'이라는 이름의 행이 필수다

    class_lst는 리스트 형태의 str으로 이루어져야한다.
    """
    if '내용' not in df.columns:
        print("내용이 없습니다")
    elif df.iloc[0,0] != 0:
        print("id(인덱스 번호)가 없습니다")
    else:
        pass

    if len(df.columns) < 3:
        df['대분류'] = ''
    else:
        pass

    ##사전 추가

    kiwi = Kiwi()

    #대분류 사전 추가
    kiwi.add_user_word('이중전공', 'DM', 3.0)
    kiwi.add_user_word('융합전공', 'UM', 3.0)
    kiwi.add_user_word('전공심화', 'MD', 3.0)
    kiwi.add_user_word('부전공', 'SM', 3.0)
    kiwi.add_user_word('연계전공', 'UM', 3.0)

    kiwi.prepare()


    ## 질문 내용 리스트화
    content = df['내용'].tolist()

    ## 형태소 분석
    sent = ''
    lem_cont = []
    for quest in content:
        ## 형태소만 떼어내기
        temp_tagging = [x[0] for x in kiwi.analyze(quest, top_n=1)]
        inner_temp = ["{}".format(word) for word, tag, score1, score2 in temp_tagging[0]]
        lem_cont.append(inner_temp)

    ## 형태소들 한 문장으로 합치기
    merge_lem = []
    for morph in lem_cont:
        txt = ' '.join(morph)
        merge_lem.append(txt)

    # 기존 데이터 프레임에 추가
    df['명사토큰'] = merge_lem

    df_id = df.iloc[:, 0].tolist()
    # 대분류
    big_class = class_lst

    quest_lst = [ [id, token] for id, token in zip(df_id, merge_lem)]

    ## 분류하기
    classified_lst = []
    for id, token in quest_lst:    
        for x in big_class:
            if x in str(token):
                df.iloc[id, 2] = x
                classified_lst.append(list(df.iloc[id]))

    ## 데이터 프레임화
    classified_df = pd.DataFrame(classified_lst)
    classified_df.columns = ['id', '내용', '분류', '명사토큰']

    ## 중복 제거
    classified_df = classified_df.drop_duplicates('id', keep='first')
    
    return classified_df

In [6]:
def merge_df(*df):
    
    # 합치기
    df_lst = list(df)
    merged_class_df = pd.concat(df_lst)

    # id 재정렬
    id_lst = []
    for i in range(0, len(merged_class_df)):
        id_lst.append(i)

    merged_class_df['id'] = id_lst

    # 칼럼 추가
    merged_class_df['Answer'] = ''
    merged_class_df['Tagset'] = ''

    # 칼럼 재정렬
    merged_class_df = merged_class_df[['id', '분류', '내용', '명사토큰', 'Answer', 'Tagset']]

    # 칼럼 이름 변경
    merged_class_df.columns = ['ID', 'CLASS', 'QUESTION', 'LEMMA', 'ANSWER', 'TAGSET']

    return merged_class_df

In [7]:
quest_df = pd.read_csv(r"E:\대학교\HOWL's\챗봇 프로젝트\질문분류by10.csv", encoding='ANSI')

eta_df = pd.read_csv(r"E:\Programming\python\NLP\HOW'Ls\Chatbot\질문.csv", encoding='ANSI')
eta_df = eta_df.drop(['글쓴이', 'id', '작성시간', '좋아요', '댓글 수'], axis=1)

dm_df = pd.read_csv(r"E:\대학교\HOWL's\챗봇 프로젝트\이중전과게시판_이중검색.csv", encoding='UTF-8')
dm_df = dm_df.drop(['댓글', '댓글쓴이', '대댓글여부', 'id'], axis=1)

In [9]:
class_quest_df = classification(quest_df)
class_eta_df = classification(eta_df)
class_dm_df = classification(dm_df)

In [12]:
merged_class_df = merge_df(class_dm_df, class_eta_df, class_quest_df)

merged_class_df.to_csv(r"E:\Programming\python\NLP\HOW'Ls\Chatbot\results\classified_question.csv", index=False, encoding="utf-8")

## 결과 파일이 깨져서 나올때는 파일을 메모장에서 열어서 다른이름으로 저장을 누르고, 인코딩을 ANSI로 바꾸고 .csv로 바꾸면된다.